<a href="https://colab.research.google.com/github/Harshkotkar/Deep-Learning/blob/main/rag_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install langchain_community
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.8 MB/s e

In [6]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 10.0 MB/s eta 0:00:00


In [8]:
#data ingestion
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("/content/kebo114.pdf")
docs=loader.load()

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_spliter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_spliter.split_documents(docs)


In [12]:
documents[:3]

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PageMaker 7.0', 'creationdate': '2022-10-14T15:17:04+00:00', 'author': 'Administrator', 'moddate': '2025-04-30T15:23:40+05:30', 'title': 'ch-14.pmd', 'source': '/content/kebo114.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='UNIT 5\nThe reductionist  approach to study of life forms resulted in increasing\nuse of physico-chemical concepts and techniques. Majority of these\nstudies employed either surviving tissue model or straightaway cell-\nfree systems. An explosion of knowledge resulted in molecular biology.\nMolecular physiology became almost synonymous with biochemistry\nand biophysics. However, it is now being incr easingly r\nealised that\nneither a purely organismic approach nor a purely reductionistic\nmolecular approach would reveal the truth about biological processes\nor living phenomena. Systems biology makes us believe that all living\nphenomena are emergent properties due to interac

In [31]:
# #vector embeding
# from langchain_community.embeddings import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma

# db=Chroma.from_documents(documents[:10],OpenAIEmbeddings())

In [20]:
#vector embeding
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Create the embedding model
model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {'device': 'cuda'}  # or 'cuda' for GPU and cpu for if no gpu avalable
encode_kwargs = {'normalize_embeddings': True}  # better for similarity search

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Create your vectorstore
db = Chroma.from_documents(documents[:10], embeddings)

In [22]:
query='who is Alfonso Corti?'
result=db.similarity_search(query)
result

[Document(metadata={'page_label': '2', 'total_pages': 12, 'author': 'Administrator', 'producer': 'GPL Ghostscript 8.15', 'creator': 'PageMaker 7.0', 'page': 1, 'title': 'ch-14.pmd', 'moddate': '2025-04-30T15:23:40+05:30', 'creationdate': '2022-10-14T15:17:04+00:00', 'source': '/content/kebo114.pdf'}, page_content='ALFONSO CORTI, Italian anatomist, was born in 1822. Corti began\nhis scientific career studying the cardiovascular systems of\nreptiles. Later, he tur ned his attention to the mammalian\nauditory system. In 1851, he published a paper describing a\nstructure located on the basilar membrane of the cochlea\ncontaining hair cells that convert sound vibrations into nerve\nimpulses, the organ of Corti. He died in the year 1888.\nAlfonso Corti\n(1822 – 1888)\nReprint 2025-26'),
 Document(metadata={'moddate': '2025-04-30T15:23:40+05:30', 'author': 'Administrator', 'page': 1, 'creator': 'PageMaker 7.0', 'source': '/content/kebo114.pdf', 'producer': 'GPL Ghostscript 8.15', 'creationdat

In [29]:
query='who is Alfonso Corti?'
result = db.similarity_search_with_relevance_scores(query)
highest_similarity = max(result, key=lambda x: x[1])
document, score = highest_similarity

print(f"Highest similarity score: {score:.4f}")
print(f"Content: {document.page_content}")

Highest similarity score: 0.5939
Content: ALFONSO CORTI, Italian anatomist, was born in 1822. Corti began
his scientific career studying the cardiovascular systems of
reptiles. Later, he tur ned his attention to the mammalian
auditory system. In 1851, he published a paper describing a
structure located on the basilar membrane of the cochlea
containing hair cells that convert sound vibrations into nerve
impulses, the organ of Corti. He died in the year 1888.
Alfonso Corti
(1822 – 1888)
Reprint 2025-26
